# ECG Processing Example

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

import biopsykit as bp
import biopsykit.signals.ecg as ecg
import biopsykit.signals.utils as utils

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')
plt.rcParams['figure.figsize'] = (10,5)
sns.set(style='ticks')

## Example 1: 1 Dataset, 1 Phase

In [ ]:
ecg_data, sampling_rate = bp.example_data.get_ecg_example()
ep = ecg.EcgProcessor(df=ecg_data, sampling_rate=sampling_rate)

### Process ECG Signal
Calling `ep.ecg_process()` will perform R peak detection and perform outlier correcrtion with the default outlier correction pipeline

In [ ]:
ep.ecg_process()

#### Optional: Use other outlier correction parameters
Calling `ep.outlier_params_default()` will list available outlier correction parameters and their default parameter. See the doumentation of `ep.outlier_corrections()` for further information.


In [ ]:
# List available outlier correction parameters and their default parameter. See the doumentation of 'EcgProcessor.outlier_corrections()' for further information.
# print(ep.outlier_params_default())
# ep.ecg_process(outlier_correction=['statistical_rr', 'statistical_rr_diff', 'physiological'], outlier_params={'statistical_rr': 2.576, 'statistical_rr_diff': 1.96, 'physiological': (50, 180)})

#### ECG and Heart Rate Result

In [ ]:
print(ep.ecg_result)

In [ ]:
# Get heart rate and print resulting heart rate 
hr_data = ep.heart_rate['Data']
hr_data.head()

In [ ]:
# Plot an overview of the acquired data
fig, axs = ecg.plotting.ecg_plot(ep, key='Data', figsize=(10,5))

### Compute HRV
Heart rate variability (HRV) is computed over the complete interval. If you want to compute HRV over different subintervals, you need to split the data first.

In [ ]:
ep.hrv_process(ep, 'Data', index='Vp01', index_name="subject_id")

#### Plot HRV Overview

In [ ]:
fig, axs = ecg.plotting.hrv_plot(ep, 'Data', figsize=(10,5))

## Example 2: 1 Dataset, Multiple Phases

In [ ]:
ecg_data, sampling_rate = bp.example_data.get_ecg_example()

For this example, we use the example ECG Data and assume we want to split it into 3 phases (names: Preparation, Stress, Recovery) of 3 minutes

In [ ]:
#provide list of edge times (start times of the phases and the total end time)
time_intervals = pd.Series(["12:32", "12:35", "12:38", "12:41"], index=["Preparation", "Stress", "Recovery", "End"])
# alternatively: provide dict with start-end times and names per phase
#time_intervals = {"Preparation": ("12:32", "12:35"), "Stress": ("12:35", "12:38"), "Recovery": ("12:38", "12:41")}

### Process all Phases

In [ ]:
ep = ecg.EcgProcessor(df=ecg_data, sampling_rate=sampling_rate, time_intervals=time_intervals)
ep.ecg_process()

#### Compute HRV parameters for each Phase
Using List Comprehension (calling `EcgProcessor.hrv_process()` for each phase) and concat the results into one dataframe using `pd.concat()`

In [ ]:
hrv_result = pd.concat([ep.hrv_process(ep, key=key, index=key) for key in ep.phases])
# alternatively: call EcgProcessor.hrv_batch_process()
# hrv_result = ep.hrv_batch_process()

In [ ]:
hrv_result

### Plot one Phase

In [ ]:
fig, axs = ecg.plotting.ecg_plot(ep, key='Stress', figsize=(10,5))

In [ ]:
fig, axs = ecg.plotting.hrv_plot(ep, key='Stress', figsize=(10,5))

## Example 3: Multiple Datasets, Multiple Phases

### Load data and time log information

In [ ]:
ecg_data, sampling_rate = bp.example_data.get_ecg_example()
ecg_data_02, sampling_rate = bp.example_data.get_ecg_example_02()

For this example, we use two ECG example datasets, where the last phase ("Recovery") differs in length

In [ ]:
subject_dict = {
    'Vp01': {
        'Data': ecg_data, 
        'Time': pd.Series(["12:32", "12:35", "12:38", "12:41"], index=["Preparation", "Stress", "Recovery", "End"])
    }, 
    'Vp02': {
        'Data': ecg_data_02,
        # The last phase of Vp02 has a length of only 1 minute to demonstrate the functionality of cutting to equal length
        'Time': pd.Series(["12:55", "12:58", "13:01", "13:02"], index=["Preparation", "Stress", "Recovery", "End"])
    }
}

### Iterate through all subjects and perform ECG processing and HRV analysis per phase

In [ ]:
from tqdm.notebook import tqdm

dict_hrv_result = {}
dict_heart_rate_result = {}

for subject_id, data_dict in tqdm(subject_dict.items(), desc="Subjects"):
    ep = ecg.EcgProcessor(df=data_dict['Data'], time_intervals=data_dict['Time'], sampling_rate=256.0)
    ep.ecg_process(title=subject_id)
    
    dict_heart_rate_result[subject_id] = ep.heart_rate
    
    hrv_result = ep.hrv_batch_process()
    dict_hrv_result[subject_id] = hrv_result

### Combine Data from all subjects

Interpolate all heart rate values to a sampling rate of 1 Hz and then cut all phases to the same length. Then, reshape the resulting dictionary into a dict with combined heart rate values of all subjects per phase

In [ ]:
dict_result = utils.interpolate_and_cut(dict_heart_rate_result)
dict_concat = utils.concat_phase_dict(dict_result, phases=["Preparation", "Stress", "Recovery"])

Concatenate the HRV result dict into one dataframe with MultiIndex

In [ ]:
hrv_result = pd.concat(dict_hrv_result)
hrv_result